# Wikipedia Kaggle Data Peak

Read dataset given by Kaggle of Wikipedia articles.

In [1]:
import sqlite3
conn = sqlite3.connect('/home/mamerisawesome/Documents/wiki-kaggle-17_18.db')
crsr = conn.cursor()

Peak into the tables.

In [2]:
crsr.execute("SELECT name FROM sqlite_master WHERE type='table';")
crsr.fetchall()

crsr.execute("SELECT * FROM sqlite_master WHERE type='table';")
res = crsr.fetchall()[0][4].split('CREATE TABLE ARTICLES (')
res = ''.join(res).split(')')
res = ''.join(res)
columns = res.split(', ')

res = []
for col in columns:
    col_split = col.split(' ')
    res += [{ "column": col_split[0], "type": col_split[1] }]
res

[{'column': 'ARTICLE_ID', 'type': 'INTEGER'},
 {'column': 'TITLE', 'type': 'TEXT'},
 {'column': 'SECTION_TITLE', 'type': 'TEXT'},
 {'column': 'SECTION_TEXT', 'type': 'TEXT'}]

In [5]:
crsr.execute("SELECT ARTICLE_ID, TITLE FROM ARTICLES GROUP BY ARTICLE_ID LIMIT 100000")
titles = crsr.fetchall()

In [6]:
parsed_titles = {}
try:
    for datum in titles:
        parsed_titles[datum[0]] = datum[1]
except KeyError:
    pass

In [7]:
{
    "length": len(parsed_titles),
    "peak": parsed_titles
}

{'length': 100000,
 'peak': {0: 'Anarchism',
  1: 'Autism',
  2: 'Albedo',
  3: 'A',
  4: 'Alabama',
  5: 'Achilles',
  6: 'Abraham Lincoln',
  7: 'Aristotle',
  8: 'An American in Paris',
  9: 'Academy Award for Best Production Design',
  10: 'Academy Awards',
  11: 'Actrius',
  12: 'Animalia (book)',
  13: 'International Atomic Time',
  14: 'Altruism',
  15: 'Ayn Rand',
  16: 'Alain Connes',
  17: 'Allan Dwan',
  18: 'Algeria',
  19: 'List of Atlas Shrugged characters',
  20: 'Anthropology',
  21: 'Agricultural science',
  22: 'Alchemy',
  23: 'Alien',
  24: 'Astronomer',
  25: 'ASCII',
  26: 'Austin (disambiguation)',
  27: 'Animation',
  28: 'Apollo',
  29: 'Andre Agassi',
  30: 'Austroasiatic languages',
  31: 'Afroasiatic languages',
  32: 'Andorra',
  33: 'Arithmetic mean',
  34: 'American Football Conference',
  35: 'Animal Farm',
  36: 'Amphibian',
  37: 'Alaska',
  38: 'Agriculture',
  39: 'Aldous Huxley',
  40: 'Ada',
  41: 'Aberdeen (disambiguation)',
  42: 'Algae',
  43: '

In [2]:
import pandas as pd
query = """
        SELECT 
            ARTICLE_ID,
            REPLACE(GROUP_CONCAT(SECTION_TEXT, ' '), '\n', ' ') AS CONTENT 
        FROM ARTICLES 
        GROUP BY ARTICLE_ID
        """

df = None
for chunk in pd.read_sql_query(query, conn, chunksize=100000):
    if df:
        df.append(chunk)
    else:
        df = chunk
    break

In [3]:
df.shape

(100000, 2)

In [18]:
df[['CONTENT']][:3]

,CONTENT
0,'''Anarchism''' is a political philosoph...
1,'''Autism''' is a neurodevelopmental ...
2,Percentage of diffusely reflected sunlight i...


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df)
idf = vectorizer.idf_

In [28]:
import re
data = df.CONTENT.values.tolist()
clean = [re.sub("[^a-zA-Z]+", " ", s) for s in data]

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import numpy as np
import gensim
import nltk
%pylab
%matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [33]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

In [34]:
vect = CountVectorizer(ngram_range=(1,1), stop_words="english")

In [35]:
dtm = vect.fit_transform(clean)

In [42]:
dtm.todense()

MemoryError: 